In [ ]:
%pip install fiftyone
# 1. Imports & Setup
import os
import sys
import json
import time
import math
import glob
import random
import copy
import shutil
from pathlib import Path
from datetime import datetime
from collections import defaultdict

import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import display, clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler

import fiftyone as fo
import fiftyone.zoo as foz

# Set Seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("✅ Notebook Updated: Version 3.0 (Fixed Collate & Criterion)")



In [ ]:
# 2. Configuration
# --- USER SETTINGS ---
QUICK_TEST = False  # Set to True for a fast smoke test (1 batch, 10 images)
BATCH_SIZE = 100 if QUICK_TEST else 2000  # Images per "roll"
NUM_BATCHES = 1 if QUICK_TEST else 10    # How many times to roll
EPOCHS_PER_BATCH = 1 if QUICK_TEST else 5 # Epochs to train on each batch

BASE_DIR = os.path.abspath("yolo-lab")
DIRS = {
    "datasets": os.path.join(BASE_DIR, "datasets"),
    "runs": os.path.join(BASE_DIR, "runs"),
    "configs": os.path.join(BASE_DIR, "configs"),
}
for d in DIRS.values():
    os.makedirs(d, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
EXP_NAME = "yolov8_local_batch"
RUN_NAME = f"{timestamp}_{EXP_NAME}"
RUN_DIR = os.path.join(DIRS["runs"], RUN_NAME)
os.makedirs(RUN_DIR, exist_ok=True)

# Model & Training Config
CFG = {
    "exp_name": EXP_NAME,
    "run_name": RUN_NAME,
    "seed": 42,
    "imgsz": 640,
    "batch_size": 8 if QUICK_TEST else 16,
    "num_classes": 80,
    
    # Model
    "width": 1.0,
    "depth": 1.0,
    "reg_max": 16,
    "head_hidden": 256,
    "backbone": "yolov8_cspdarknet",
    
    # Optimizer
    "optimizer": "adamw",
    "lr": 1e-3,
    "weight_decay": 0.05,
    "cosine_schedule": True,
    "epochs": EPOCHS_PER_BATCH, # Per batch
    "amp": True,
    "grad_clip_norm": 10.0,
    "ema_decay": 0.9998,
    
    # Loss
    "tal_alpha": 1.0,
    "tal_beta": 6.0,
    "tal_topk": 10,
    "tal_center_radius": 2.5,
    "loss_weights": {"box": 7.5, "cls": 0.5, "dfl": 1.5}, # Adjusted for v8
    
    # Augmentation
    "letterbox_pad": 114,
    "hflip_p": 0.5,
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    
    # Paths (Dynamic per batch)
    "data_root": os.path.join(DIRS["datasets"], "current_batch"),
    "train_img_dir": "images/train",
    "train_lbl_dir": "labels/train",
    "val_img_dir": "images/val",
    "val_lbl_dir": "labels/val",
}

print("Run Directory:", RUN_DIR)



In [ ]:
# 3. Model Architecture
def autopad(k, p=None, d=1):
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]
    return p

class Conv(nn.Module):
    default_act = nn.SiLU()
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class Bottleneck(nn.Module):
    def __init__(self, c1, c2, shortcut=True, g=1, k=(3, 3), e=0.5):
        super().__init__()
        c_ = int(c2 * e)
        self.cv1 = Conv(c1, c_, k[0], 1)
        self.cv2 = Conv(c_, c2, k[1], 1, g=g)
        self.add = shortcut and c1 == c2
    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))

class C2f(nn.Module):
    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):
        super().__init__()
        self.c = int(c2 * e)
        self.cv1 = Conv(c1, 2 * self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3)), e=1.0) for _ in range(n))
    def forward(self, x):
        y = list(self.cv1(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))

class SPPF(nn.Module):
    def __init__(self, c1, c2, k=5):
        super().__init__()
        c_ = c1 // 2
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)
    def forward(self, x):
        x = self.cv1(x)
        y1 = self.m(x)
        y2 = self.m(y1)
        return self.cv2(torch.cat((x, y1, y2, self.m(y2)), 1))

class CSPDarknet(nn.Module):
    def __init__(self, width=1.0, depth=1.0):
        super().__init__()
        base_c = [64, 128, 256, 512, 1024]
        base_d = [3, 6, 6, 3]
        self.c = [int(x * width) for x in base_c]
        self.d = [max(round(x * depth), 1) if x > 1 else x for x in base_d]
        self.stem = Conv(3, self.c[0], 3, 2)
        self.stage1 = nn.Sequential(Conv(self.c[0], self.c[1], 3, 2), C2f(self.c[1], self.c[1], n=self.d[0], shortcut=True))
        self.stage2 = nn.Sequential(Conv(self.c[1], self.c[2], 3, 2), C2f(self.c[2], self.c[2], n=self.d[1], shortcut=True))
        self.stage3 = nn.Sequential(Conv(self.c[2], self.c[3], 3, 2), C2f(self.c[3], self.c[3], n=self.d[2], shortcut=True))
        self.stage4 = nn.Sequential(Conv(self.c[3], self.c[4], 3, 2), C2f(self.c[4], self.c[4], n=self.d[3], shortcut=True), SPPF(self.c[4], self.c[4], k=5))

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        c3 = self.stage2(x)
        c4 = self.stage3(c3)
        c5 = self.stage4(c4)
        return c3, c4, c5

class YOLOv8PAFPN(nn.Module):
    def __init__(self, c3, c4, c5, out_ch=256, width=1.0, depth=1.0):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        self.reduce5 = Conv(c5, c4, 1, 1)
        self.c2f_p4 = C2f(c4 + c4, c4, n=3, shortcut=False)
        self.reduce4 = Conv(c4, c3, 1, 1)
        self.c2f_p3 = C2f(c3 + c3, c3, n=3, shortcut=False)
        self.down3 = Conv(c3, c3, 3, 2)
        self.c2f_n4 = C2f(c3 + c4, c4, n=3, shortcut=False)
        self.down4 = Conv(c4, c4, 3, 2)
        self.c2f_n5 = C2f(c4 + c5, c5, n=3, shortcut=False)

    def forward(self, c3, c4, c5):
        p5 = c5
        p4 = self.reduce5(p5)
        p4_out = self.c2f_p4(torch.cat([self.up(p4), c4], dim=1))
        p3 = self.reduce4(p4_out)
        p3_out = self.c2f_p3(torch.cat([self.up(p3), c3], dim=1))
        n3 = p3_out
        n4_out = self.c2f_n4(torch.cat([self.down3(n3), p4_out], dim=1))
        n5_out = self.c2f_n5(torch.cat([self.down4(n4_out), p5], dim=1))
        return n3, n4_out, n5_out

class Integral(nn.Module):
    def __init__(self, reg_max=16):
        super().__init__()
        self.reg_max = int(reg_max)
        self.register_buffer("proj", torch.arange(self.reg_max + 1, dtype=torch.float32), persistent=False)
    def forward(self, logits):
        return (logits.softmax(dim=-1) * self.proj).sum(dim=-1)

class YoloV8LiteHead(nn.Module):
    def __init__(self, in_channels_list, num_classes=80, hidden=256, reg_max=16):
        super().__init__()
        self.num_classes = num_classes
        self.reg_max = reg_max
        self.integral = Integral(self.reg_max)
        self.cls_towers = nn.ModuleList()
        self.reg_towers = nn.ModuleList()
        self.cls_preds = nn.ModuleList()
        self.box_preds = nn.ModuleList()
        
        for in_ch in in_channels_list:
            self.cls_towers.append(nn.Sequential(Conv(in_ch, hidden, 3, 1), Conv(hidden, hidden, 3, 1)))
            self.reg_towers.append(nn.Sequential(Conv(in_ch, hidden, 3, 1), Conv(hidden, hidden, 3, 1)))
            self.cls_preds.append(nn.Conv2d(hidden, num_classes, 1))
            self.box_preds.append(nn.Conv2d(hidden, 4 * (self.reg_max + 1), 1))

    def forward(self, features):
        cls_outs = []
        box_outs = []
        for i, f in enumerate(features):
            cls_outs.append(self.cls_preds[i](self.cls_towers[i](f)))
            box_outs.append(self.box_preds[i](self.reg_towers[i](f)))
        return cls_outs, box_outs

class YoloModel(nn.Module):
    def __init__(self, num_classes=80, backbone="yolov8_cspdarknet", head_hidden=256, fpn_out=256):
        super().__init__()
        width = CFG.get("width", 1.0)
        depth = CFG.get("depth", 1.0)
        self.backbone = CSPDarknet(width=width, depth=depth)
        base_c = [256, 512, 1024]
        c3, c4, c5 = [int(x * width) for x in base_c]
        self.neck = YOLOv8PAFPN(c3=c3, c4=c4, c5=c5, out_ch=fpn_out, width=width, depth=depth)
        self.head = YoloV8LiteHead(in_channels_list=[c3, c4, c5], num_classes=num_classes, hidden=head_hidden, reg_max=CFG.get("reg_max", 16))
        self.strides = [8, 16, 32]

    def forward(self, x, targets=None):
        c3, c4, c5 = self.backbone(x)
        p3, p4, p5 = self.neck(c3, c4, c5)
        cls_outs, box_outs = self.head([p3, p4, p5])
        head_out = {"features": [p3, p4, p5], "cls": cls_outs, "box": box_outs, "strides": self.strides}
        
        if self.training and targets is not None and hasattr(self, "criterion"):
            losses, stats = self.criterion(head_out, targets)
            return losses, stats
        return head_out



In [ ]:
# 4. Utils & Loss
def make_grid(h, w, stride, device):
    ys = torch.arange(h, device=device)
    xs = torch.arange(w, device=device)
    yy, xx = torch.meshgrid(ys, xs, indexing="ij")
    cx = (xx + 0.5) * stride
    cy = (yy + 0.5) * stride
    return cx.reshape(-1), cy.reshape(-1)

def box_iou_xyxy_matrix(a, b):
    if a.numel() == 0 or b.numel() == 0: return a.new_zeros((a.shape[0], b.shape[0]))
    area_a = ((a[:, 2] - a[:, 0]).clamp(min=0) * (a[:, 3] - a[:, 1]).clamp(min=0))[:, None]
    area_b = ((b[:, 2] - b[:, 0]).clamp(min=0) * (b[:, 3] - b[:, 1]).clamp(min=0))[None, :]
    x1 = torch.maximum(a[:, None, 0], b[None, :, 0])
    y1 = torch.maximum(a[:, None, 1], b[None, :, 1])
    x2 = torch.minimum(a[:, None, 2], b[None, :, 2])
    y2 = torch.minimum(a[:, None, 3], b[None, :, 3])
    inter = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
    return inter / (area_a + area_b - inter + 1e-6)

def bbox_iou(box1, box2, eps=1e-7):
    # box1: [N, 4], box2: [N, 4]
    b1_x1, b1_y1, b1_x2, b1_y2 = box1.chunk(4, -1)
    b2_x1, b2_y1, b2_x2, b2_y2 = box2.chunk(4, -1)
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)
    union = w1 * h1 + w2 * h2 - inter + eps
    return inter / union

class DetectionLoss(nn.Module):
    def __init__(self, num_classes, image_size, strides, lambda_box=7.5, lambda_cls=0.5, dfl_ch=17):
        super().__init__()
        self.nc = num_classes
        self.imgsz = image_size
        self.strides = strides
        self.lambda_box = lambda_box
        self.lambda_cls = lambda_cls
        self.dfl_ch = dfl_ch
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, head_out, targets):
        cls_outs = head_out["cls"]
        box_outs = head_out["box"]
        
        targets_per_image, levels = self.build_targets(cls_outs, box_outs, targets)
        
        loss_cls = torch.tensor(0.0, device=cls_outs[0].device)
        loss_box = torch.tensor(0.0, device=cls_outs[0].device)
        num_pos_total = 0.0
        
        for b in range(len(targets_per_image)):
            t = targets_per_image[b]
            pos_mask = t["pos_index"]
            num_pos = len(pos_mask)
            num_pos_total += num_pos
            
            # Classification Loss
            pred_cls = torch.cat([c[b].permute(1,2,0).reshape(-1, self.nc) for c in cls_outs], 0)
            t_cls = torch.zeros_like(pred_cls)
            if num_pos > 0:
                t_cls[pos_mask] = t["t_cls_soft"].to(t_cls.dtype)
            
            l_cls = self.bce(pred_cls, t_cls).sum()
            loss_cls += l_cls
            
            # Box Loss (IoU + DFL)
            if num_pos > 0:
                pred_box_dist = torch.cat([x[b].permute(1,2,0).reshape(-1, 4 * self.dfl_ch) for x in box_outs], 0)
                pred_box_pos = pred_box_dist[pos_mask].view(-1, 4, self.dfl_ch)
                
                # DFL
                t_box_ltrb = t["t_box_ltrb"]
                tl = t_box_ltrb.long()
                tr = tl + 1
                wl = tr.float() - t_box_ltrb
                wr = t_box_ltrb - tl.float()
                
                l_dfl = (F.cross_entropy(pred_box_pos.view(-1, self.dfl_ch), tl.view(-1), reduction="none").view(-1, 4) * wl +
                         F.cross_entropy(pred_box_pos.view(-1, self.dfl_ch), tr.view(-1), reduction="none").view(-1, 4) * wr).mean(-1)
                
                # IoU
                # Decode box for IoU
                anchors_cx, anchors_cy = [], []
                for level_i, level in enumerate(levels):
                    cx, cy = make_grid(level["H"], level["W"], level["stride"], pred_box_dist.device)
                    anchors_cx.append(cx)
                    anchors_cy.append(cy)
                anchors_cx = torch.cat(anchors_cx)[pos_mask]
                anchors_cy = torch.cat(anchors_cy)[pos_mask]
                
                pred_ltrb = (pred_box_pos.softmax(dim=-1) * torch.arange(self.dfl_ch, device=pred_box_pos.device).float()).sum(dim=-1)
                
                pred_x1 = anchors_cx - pred_ltrb[:, 0]
                pred_y1 = anchors_cy - pred_ltrb[:, 1]
                pred_x2 = anchors_cx + pred_ltrb[:, 2]
                pred_y2 = anchors_cy + pred_ltrb[:, 3]
                pred_xyxy = torch.stack([pred_x1, pred_y1, pred_x2, pred_y2], dim=-1)
                
                iou = bbox_iou(pred_xyxy, t["t_box_xyxy"])
                l_iou = 1.0 - iou
                
                loss_box += (l_iou + l_dfl).sum() * self.lambda_box

        # Normalize
        norm = max(num_pos_total, 1.0)
        loss_cls = (loss_cls * self.lambda_cls) / norm
        loss_box = loss_box / norm
        
        return {"loss": loss_cls + loss_box, "loss_cls": loss_cls, "loss_box": loss_box}, {"num_pos": num_pos_total}

    def build_targets(self, cls_outs, box_outs, targets):
        gt_classes = targets["labels"]
        gt_boxes = targets["boxes"]
        batch_idx = targets["batch_index"]
        
        B = cls_outs[0].shape[0]
        gt_cls_list = []
        gt_box_list = []
        for i in range(B):
            mask = batch_idx == i
            gt_cls_list.append(gt_classes[mask])
            gt_box_list.append(gt_boxes[mask])
            
        return build_targets_task_aligned(cls_outs, box_outs, self.strides, gt_cls_list, gt_box_list, self.imgsz)

def build_targets_task_aligned(cls_outs, box_outs, strides, gt_classes, gt_boxes_xyxy, image_size):
    device = cls_outs[0].device
    B = cls_outs[0].shape[0]
    C = cls_outs[0].shape[1]
    
    levels = []
    start = 0
    grids = []
    
    for (cl, s) in zip(cls_outs, strides):
        _, _, H, W = cl.shape
        levels.append({"H": H, "W": W, "stride": s, "start": start, "end": start + H * W})
        cx, cy = make_grid(H, W, s, device)
        grids.append((cx, cy))
        start += H * W
        
    tal_alpha = float(CFG.get("tal_alpha", 1.0))
    tal_beta = float(CFG.get("tal_beta", 6.0))
    tal_topk = int(CFG.get("tal_topk", 10))
    tal_cr = float(CFG.get("tal_center_radius", 2.5))
    
    per_image_targets = []
    for b in range(B):
        cls_per_image = [cl[b].permute(1, 2, 0).reshape(-1, C) for cl in cls_outs]
        cls_flat = torch.cat(cls_per_image, dim=0)
        N_total = cls_flat.shape[0]
        
        gtc = gt_classes[b]
        gtb = gt_boxes_xyxy[b]
        Ng = int(gtc.numel())
        
        if Ng == 0:
            per_image_targets.append({
                "t_cls_soft": torch.zeros(0, C, device=device),
                "t_box_xyxy": torch.zeros(0, 4, device=device),
                "t_box_ltrb": torch.zeros(0, 4, device=device),
                "pos_index": torch.zeros(0, dtype=torch.long, device=device),
            })
            continue
        
        pred_xyxy_levels = []
        for (bx, level, (cx, cy)) in zip(box_outs, levels, grids):
            H, W, s = level["H"], level["W"], level["stride"]
            bl = bx[b]
            M1 = bl.shape[0] // 4
            bl = bl.view(4, M1, H, W).permute(2, 3, 0, 1).reshape(H * W, 4, M1)
            probs = bl.softmax(dim=-1)
            proj = torch.arange(M1, device=device, dtype=bl.dtype)
            dists = (probs * proj).sum(dim=-1) * float(s)
            
            x1 = cx - dists[:, 0]
            y1 = cy - dists[:, 1]
            x2 = cx + dists[:, 2]
            y2 = cy + dists[:, 3]
            pred_xyxy_levels.append(torch.stack([x1, y1, x2, y2], dim=-1).clamp_(0, image_size))
            
        pred_xyxy = torch.cat(pred_xyxy_levels, dim=0)

        candidate_mask = torch.zeros(N_total, Ng, dtype=torch.bool, device=device)
        for level, (cx, cy) in enumerate(grids):
            start, end, s = levels[level]["start"], levels[level]["end"], levels[level]["stride"]
            Nl = end - start
            cxv, cyv = cx.view(Nl, 1), cy.view(Nl, 1)
            
            if tal_cr > 0:
                gt_centers = 0.5 * (gtb[:, :2] + gtb[:, 2:])
                half = tal_cr * s
                in_center = (cxv >= gt_centers[:, 0] - half) & (cyv >= gt_centers[:, 1] - half) & \
                            (cxv <= gt_centers[:, 0] + half) & (cyv <= gt_centers[:, 1] + half)
                candidate_mask[start:end] |= in_center
            else:
                in_box = (cxv >= gtb[:, 0]) & (cyv >= gtb[:, 1]) & (cxv <= gtb[:, 2]) & (cyv <= gtb[:, 3])
                candidate_mask[start:end] |= in_box
                
        cls_sigmoid = cls_flat.sigmoid()
        cls_gt_scores = cls_sigmoid[:, gtc]
        iou_matrix = box_iou_xyxy_matrix(pred_xyxy, gtb)
        align = (cls_gt_scores.clamp(min=1e-9).pow(tal_alpha)) * (iou_matrix.clamp(min=1e-9).pow(tal_beta))
        align = torch.where(candidate_mask, align, torch.full_like(align, -1e-9))
        
        k = min(tal_topk, align.shape[0])
        topk_scores, topk_index = torch.topk(align, k, dim=0)
        
        best_gt_per_pred = torch.full((N_total,), -1, dtype=torch.long, device=device)
        best_score_per_pred = torch.full((N_total,), -1e-9, dtype=align.dtype, device=device)
        
        for j in range(Ng):
            idx_j = topk_index[:, j]
            score_j = topk_scores[:, j]
            better = score_j > best_score_per_pred[idx_j]
            best_gt_per_pred[idx_j[better]] = j
            best_score_per_pred[idx_j[better]] = score_j[better]
            
        pos_mask = best_gt_per_pred >= 0
        pos_index = torch.nonzero(pos_mask, as_tuple=False).squeeze(1)
        
        if pos_index.numel() == 0:
            per_image_targets.append({
                "t_cls_soft": torch.zeros(0, C, device=device),
                "t_box_xyxy": torch.zeros(0, 4, device=device),
                "t_box_ltrb": torch.zeros(0, 4, device=device),
                "pos_index": pos_index,
            })
            continue
        
        gt_index = best_gt_per_pred[pos_index]
        scores = best_score_per_pred[pos_index].clamp(min=0.0)
        t_cls_soft = torch.zeros(len(pos_index), C, device=device)
        t_cls_soft[torch.arange(len(pos_index)), gtc[gt_index]] = scores
        t_box_xyxy = gtb[gt_index]
        
        t_box_ltrb = torch.empty(len(pos_index), 4, device=device)
        for level_i, level in enumerate(levels):
            start, end, s = level["start"], level["end"], level["stride"]
            cx, cy = grids[level_i]
            in_level = (pos_index >= start) & (pos_index < end)
            if in_level.any():
                idx_l = pos_index[in_level] - start
                ct = torch.stack((cx[idx_l], cy[idx_l]), dim=-1)
                gs = gtb[gt_index[in_level]]
                t_box_ltrb[in_level] = torch.stack((ct[:,0]-gs[:,0], ct[:,1]-gs[:,1], gs[:,2]-ct[:,0], gs[:,3]-ct[:,1]), dim=-1).clamp(min=0, max=float(image_size))

        per_image_targets.append({
            "t_cls_soft": t_cls_soft,
            "t_box_xyxy": t_box_xyxy,
            "t_box_ltrb": t_box_ltrb,
            "pos_index": pos_index,
        })
    
    return per_image_targets, levels

# --- Visualization Utils ---
def decode_outputs(head_out, strides, conf_thres=0.25, iou_thres=0.45, max_det=300):
    # head_out: dict with 'cls' and 'box'
    device = head_out["cls"][0].device
    cls_outs = head_out["cls"]
    box_outs = head_out["box"]
    B = cls_outs[0].shape[0]
    
    preds = []
    
    # Generate grids
    grids = []
    for (cl, s) in zip(cls_outs, strides):
        _, _, H, W = cl.shape
        cx, cy = make_grid(H, W, s, device)
        grids.append((cx, cy, s))
        
    for b in range(B):
        # Collect all predictions for this image
        batch_boxes = []
        batch_scores = []
        batch_classes = []
        
        for i, (bx, cx, cy, s) in enumerate(zip(box_outs, *zip(*grids))):
            # Box decoding
            bl = bx[b] # [4*reg_max, H, W]
            C_box = bl.shape[0]
            reg_max = C_box // 4
            H, W = bl.shape[1], bl.shape[2]
            
            bl = bl.view(4, reg_max, H*W).permute(2, 0, 1) # [HW, 4, reg_max]
            probs = bl.softmax(dim=-1)
            proj = torch.arange(reg_max, device=device, dtype=bl.dtype)
            ltrb = (probs * proj).sum(dim=-1) * s # [HW, 4]
            
            x1 = cx - ltrb[:, 0]
            y1 = cy - ltrb[:, 1]
            x2 = cx + ltrb[:, 2]
            y2 = cy + ltrb[:, 3]
            xyxy = torch.stack([x1, y1, x2, y2], dim=-1)
            
            # Class scores
            cl = cls_outs[i][b].permute(1, 2, 0).reshape(-1, cls_outs[i].shape[1]) # [HW, nc]
            scores = cl.sigmoid()
            
            # Filter by conf
            max_scores, max_classes = scores.max(dim=1)
            mask = max_scores > conf_thres
            
            if mask.any():
                batch_boxes.append(xyxy[mask])
                batch_scores.append(max_scores[mask])
                batch_classes.append(max_classes[mask])
                
        if not batch_boxes:
            preds.append(torch.zeros(0, 6, device=device))
            continue
            
        batch_boxes = torch.cat(batch_boxes, dim=0)
        batch_scores = torch.cat(batch_scores, dim=0)
        batch_classes = torch.cat(batch_classes, dim=0)
        
        # NMS
        keep = torch.ops.torchvision.nms(batch_boxes, batch_scores, iou_thres)
        keep = keep[:max_det]
        
        # [x1, y1, x2, y2, conf, cls]
        out = torch.cat([batch_boxes[keep], batch_scores[keep, None], batch_classes[keep, None].float()], dim=1)
        preds.append(out)
        
    return preds

def visualize_batch_results(model, loader, device, run_dir, batch_idx, num_samples=3):
    model.eval()
    images, targets = next(iter(loader))
    images = images.to(device)
    
    with torch.no_grad():
        head_out = model(images)
        preds = decode_outputs(head_out, model.strides)
        
    # Plot
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    if num_samples == 1: axes = [axes]
    
    for i in range(min(num_samples, len(images))):
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = np.ascontiguousarray(img) * 255
        img = img.astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # GT
        gt_mask = targets["batch_index"] == i
        gt_boxes = targets["boxes"][gt_mask]
        for box in gt_boxes:
            x1, y1, x2, y2 = map(int, box.tolist())
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green
            
        # Pred
        pred = preds[i]
        for p in pred:
            x1, y1, x2, y2, conf, cls = p.tolist()
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2) # Red
            cv2.putText(img, f"{conf:.2f}", (int(x1), int(y1)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            
        if num_samples > 1:
            axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i].axis("off")
            axes[i].set_title(f"Sample {i}")
        else:
            axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[0].axis("off")
            axes[0].set_title(f"Sample {i}")
        
    plt.tight_layout()
    save_path = os.path.join(run_dir, f"val_batch_{batch_idx}.png")
    plt.savefig(save_path)
    plt.close()
    print(f"🖼️ Visualization saved to {save_path}")
    model.train()


In [ ]:
# 5. Dataset & Dataloader
class YoloDataset(Dataset):
    def __init__(self, image_dir, label_dir, imgsz=640, augment=True, pad_value=114):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.imgsz = imgsz
        self.augment = augment
        self.pad_value = pad_value
        
        # Support multiple extensions
        self.image_paths = []
        for ext in ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.JPG", "*.JPEG", "*.PNG"):
            self.image_paths.extend(glob.glob(os.path.join(image_dir, ext)))
        self.image_paths = sorted(self.image_paths)
        
        self.label_paths = [os.path.join(label_dir, Path(p).stem + ".txt") for p in self.image_paths]
        
        if len(self.image_paths) == 0:
            print(f"⚠️ WARNING: No images found in {image_dir}")
            print(f"   Did the export work? Check {os.path.dirname(image_dir)}")
        else:
            print(f"✅ Loaded {len(self.image_paths)} images from {image_dir}")
        
    def __len__(self): return len(self.image_paths)
    
    def __getitem__(self, index):
        img = cv2.imread(self.image_paths[index])
        h, w = img.shape[:2]
        
        # Read labels
        lbl_path = self.label_paths[index]
        boxes = []
        cls = []
        if os.path.exists(lbl_path):
            with open(lbl_path) as f:
                for line in f:
                    parts = list(map(float, line.strip().split()))
                    if len(parts) == 5:
                        cls.append(int(parts[0]))
                        # YOLO xywh to xyxy
                        cx, cy, bw, bh = parts[1:]
                        x1 = (cx - bw/2) * w
                        y1 = (cy - bh/2) * h
                        x2 = (cx + bw/2) * w
                        y2 = (cy + bh/2) * h
                        boxes.append([x1, y1, x2, y2])
        
        boxes = torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4))
        cls = torch.tensor(cls, dtype=torch.long) if cls else torch.zeros((0,), dtype=torch.long)
        
        # Letterbox (Simplified)
        r = min(self.imgsz / h, self.imgsz / w)
        nw, nh = int(w * r), int(h * r)
        img = cv2.resize(img, (nw, nh))
        
        # Pad
        pad_w = self.imgsz - nw
        pad_h = self.imgsz - nh
        img = cv2.copyMakeBorder(img, pad_h//2, pad_h-pad_h//2, pad_w//2, pad_w-pad_w//2, cv2.BORDER_CONSTANT, value=(114,114,114))
        
        # Adjust boxes
        if len(boxes):
            boxes[:, [0, 2]] = boxes[:, [0, 2]] * r + pad_w//2
            boxes[:, [1, 3]] = boxes[:, [1, 3]] * r + pad_h//2
            
        img = torch.from_numpy(img.transpose(2, 0, 1)).float() / 255.0
        
        target = {
            "boxes": boxes,
            "labels": cls,
            "image_id": Path(self.image_paths[index]).stem,
            "orig_size": (h, w),
            "scale": r,
            "pad": (pad_w//2, pad_h//2)
        }
        return img, target

def collate_fn(batch):
    images, targets = list(zip(*batch))
    images = torch.stack(images, dim=0)
    
    all_boxes = []
    all_labels = []
    all_bidx = []
    image_ids = []
    scales = []
    pads = []
    orig_sizes = []
    
    for i, t in enumerate(targets):
        n = t["boxes"].shape[0]
        if n:
            all_boxes.append(t["boxes"])
            all_labels.append(t["labels"])
            all_bidx.append(torch.full((n,), i, dtype=torch.long))
            
        image_ids.append(t["image_id"])
        scales.append(t["scale"])
        pads.append(t["pad"])
        orig_sizes.append(t["orig_size"])
        
    if len(all_boxes):
        boxes = torch.cat(all_boxes, 0)
        labels = torch.cat(all_labels, 0)
        bidx = torch.cat(all_bidx, 0)
    else:
        boxes = torch.zeros((0, 4), dtype=torch.float32)
        labels = torch.zeros((0,), dtype=torch.long)
        bidx = torch.zeros((0,), dtype=torch.long)
        
    return images, {
        "boxes": boxes,
        "labels": labels,
        "batch_index": bidx,
        "image_id": image_ids,
        "scale": scales,
        "pad": pads,
        "orig_size": orig_sizes
    }



In [ ]:
# 6. Batch Management Logic
def prepare_batch(batch_idx, size=2000):
    print(f"\n📦 Preparing Batch {batch_idx} (Size: {size})...")
    
    current_batch_name = f"batch_{batch_idx}"
    
    # 1. Safe Cleanup of Old Batches
    if foz.list_datasets():
        for name in foz.list_datasets():
            if name.startswith("batch_") and name != current_batch_name:
                print(f"🧹 Deleting old dataset: {name}")
                fo.delete_dataset(name)
    
    # Force delete current if exists to ensure fresh download/export
    if current_batch_name in foz.list_datasets():
         print(f"♻️  Deleting existing {current_batch_name} to ensure fresh start")
         fo.delete_dataset(current_batch_name)

    # 2. Download/Load from Zoo
    dataset = foz.load_zoo_dataset(
        "coco-2017",
        split="train", 
        label_types=["detections"],
        max_samples=size,
        shuffle=True,
        seed=batch_idx * 999, 
        dataset_name=current_batch_name,
        drop_existing=True 
    )
    
    # 3. Export to YOLO format
    out_dir = CFG["data_root"]
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir) 
        
    dataset.export(
        export_dir=out_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
    )
    print(f"✅ Batch {batch_idx} exported to {out_dir}")
    return dataset

def cleanup_batch(dataset):
    print("🧹 Cleaning up batch...")
    dataset.delete()
    if os.path.exists(CFG["data_root"]):
        shutil.rmtree(CFG["data_root"])
    print("✨ Cleanup complete.")


In [ ]:
# 7. Main Execution Loop
state_file = os.path.join(DIRS["runs"], "checkpoint_state.json")
last_ckpt_path = os.path.join(RUN_DIR, "last.pt")

# --- Initialization (ONCE) ---
model = YoloModel(num_classes=CFG["num_classes"]).to(device)
criterion = DetectionLoss(
    num_classes=CFG["num_classes"],
    image_size=CFG["imgsz"],
    strides=[8, 16, 32],
    lambda_box=CFG["loss_weights"]["box"],
    lambda_cls=CFG["loss_weights"]["cls"]
)
model.criterion = criterion
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"])
scaler = GradScaler(enabled=CFG["amp"])

# --- Resume Logic ---
start_batch = 0
start_epoch = 0
global_step = 0

if os.path.exists(last_ckpt_path):
    print(f"🔄 Resuming from {last_ckpt_path}")
    ckpt = torch.load(last_ckpt_path)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    scaler.load_state_dict(ckpt["scaler"])
    
    # Restore state
    start_batch = ckpt.get("batch_idx", 0)
    start_epoch = ckpt.get("epoch", 0)
    global_step = ckpt.get("global_step", 0)
    
    # If we finished the last batch completely, move to next
    if ckpt.get("batch_complete", False):
        start_batch += 1
        start_epoch = 0
        
    print(f"   -> Batch: {start_batch}, Epoch: {start_epoch}, Step: {global_step}")

# --- Helper: Save Checkpoint ---
def save_checkpoint(path, batch_idx, epoch, step, complete=False):
    torch.save({
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict(),
        "batch_idx": batch_idx,
        "epoch": epoch,
        "global_step": step,
        "batch_complete": complete
    }, path)
    # Also save state json for easy reading
    with open(state_file, "w") as f:
        json.dump({"batch_idx": batch_idx, "epoch": epoch, "global_step": step}, f)

# --- Main Loop ---
try:
    for b_idx in range(start_batch, NUM_BATCHES):
        print(f"\n=== STARTING BATCH {b_idx + 1}/{NUM_BATCHES} === ")
        
        # 1. Prepare Data
        ds = prepare_batch(b_idx, size=BATCH_SIZE)
        
        # Detect images folder
        if os.path.isdir(os.path.join(CFG["data_root"], "images/train")):
            train_img_path = os.path.join(CFG["data_root"], "images/train")
            train_lbl_path = os.path.join(CFG["data_root"], "labels/train")
        elif os.path.isdir(os.path.join(CFG["data_root"], "images/val")):
            print("⚠️ Using images/val as training source")
            train_img_path = os.path.join(CFG["data_root"], "images/val")
            train_lbl_path = os.path.join(CFG["data_root"], "labels/val")
        else:
            # Maybe root?
            train_img_path = CFG["data_root"]
            train_lbl_path = CFG["data_root"] 
            
        print(f"   -> Training data: {train_img_path}")
        
        train_ds = YoloDataset(train_img_path, train_lbl_path, imgsz=CFG["imgsz"])
        if len(train_ds) == 0:
            print("❌ Dataset empty. Skipping.")
            cleanup_batch(ds)
            continue
            
        train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], shuffle=True, collate_fn=collate_fn)
        
        # 2. Train Epochs
        # If resuming mid-batch, start_epoch applies only to the first batch
        current_start_epoch = start_epoch if b_idx == start_batch else 0
        
        for epoch in range(current_start_epoch, CFG["epochs"]):
            model.train()
            epoch_loss = 0.0
            num_pos_sum = 0
            
            for i, (imgs, targets) in enumerate(train_loader):
                imgs = imgs.to(device)
                for k, v in targets.items():
                    if isinstance(v, torch.Tensor):
                        targets[k] = v.to(device)
                
                optimizer.zero_grad(set_to_none=True)
                
                with torch.amp.autocast("cuda", enabled=CFG["amp"]):
                    head_out = model(imgs)
                    losses, stats = model.criterion(head_out, targets)
                    
                    total_loss = losses["loss"]
                
                scaler.scale(total_loss).backward()
                
                if CFG.get("grad_clip_norm"):
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), CFG["grad_clip_norm"])
                    
                scaler.step(optimizer)
                scaler.update()
                
                # Logging
                epoch_loss += total_loss.item()
                num_pos_sum += stats["num_pos"]
                global_step += 1
                
                if i % 10 == 0:
                    print(f"Batch {b_idx+1} Ep {epoch+1} It {i}: Loss {total_loss.item():.4f} (Box {losses['loss_box']:.4f} Cls {losses['loss_cls']:.4f}) Pos {stats['num_pos']}")
                
                # Intra-epoch Checkpoint
                if i > 0 and i % 200 == 0:
                    save_checkpoint(last_ckpt_path, b_idx, epoch, global_step)
                    
            # End of Epoch
            avg_loss = epoch_loss / len(train_loader)
            print(f"✅ Epoch {epoch+1} Complete. Avg Loss: {avg_loss:.4f} Avg Pos: {num_pos_sum/len(train_loader):.1f}")
            
            # Save Checkpoint
            save_checkpoint(last_ckpt_path, b_idx, epoch + 1, global_step) # epoch+1 so we resume next
            
        # 3. End of Batch
        print(f"🎉 Batch {b_idx+1} Complete!")
        save_checkpoint(last_ckpt_path, b_idx, CFG["epochs"], global_step, complete=True)
        
        # Visualization
        visualize_batch_results(model, train_loader, device, RUN_DIR, b_idx)
        
        # Cleanup
        cleanup_batch(ds)
        
except KeyboardInterrupt:
    print("\n🛑 Interrupted! Saving state...")
    save_checkpoint(os.path.join(RUN_DIR, "interrupted.pt"), b_idx, epoch, global_step)
    print("Saved interrupted.pt")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    save_checkpoint(os.path.join(RUN_DIR, "error_state.pt"), b_idx, epoch, global_step)
    raise e


    Batch 4 Ep 1 It 80: Loss 0.0000 (Box 0.0000 Cls 0.0000)
